# Pre-processing Notebook

In [5]:
import pandas as pd
import numpy as np
import scipy
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from scipy.stats import norm
from scipy.stats import chisquare
from scipy.stats import chi2_contingency

%matplotlib inline

In [6]:
# Avoid runtime error messages
pd.set_option('display.float_format', lambda x:'%f'%x)

# Gather

* What process generates this data?
* Is it generated from industrial equipment, a website, internal software?
* When was it created?
* How often is it updated?
* What database(if any) is it stored in?
* Who are the admins of the database?
* Can you view the schema?
* What is the process that the raw data has gone through before it reached your hands? Has it already been pre-processed before it reaches you?
* Is there a data dictionary describing every column?
* What systems use the data?
* Have their been previous data scientists working with this dataset?
* How has data changed over time? Which columns have been added/subtracted? 
* Is data for some columns not being collected?
#### Subject Matter Research
* Read articles, watch videos, talk to local subject matter experts
* Read articles/papers by academics who have already studied the field using statistical analysis
* Could be beneficial to do some analysis first as to not bias your results

- Create a folder in your file system to hold all your files for the analysis
- Create a documents/spreadsheets file to store the names, titles, contact information and notes of all the people connected to your data
- Find and introduce yourself to all the people connected to your data
- Connections to others is key to making your projects work. The more you are visible to others the more information will freely pass your way

## Source: Files -- Reading various filetypes

In [ ]:
# Zip File
import zipfile
# Extract all contents from zip file
with zipfile.ZipFile('armenian-online-job-postings.zip', 'r') as myzip:
    myzip.extractall()

In [ ]:
# CSV
df = pd.read_csv('patients.csv')

In [ ]:
# Reading TSV files
df = pd.read_csv('bestofrt.tsv',sep='\t')

In [ ]:
# JSON
df = pd.read_json('patients.csv')

In [ ]:
# Pickle file
df = pd.read_pickle('adataset.pkl')

## Character Encodings

Most files you'll encounter will probably be encoded with UTF-8. This is what Python expects by default, so most of the time you won't run into problems. However, sometimes you'll get an error.

*UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 11: invalid start byte*

Use the chardet module to try and automatically guess what the right encoding is. It's not 100% guaranteed to be right, but it's usually faster than just trying to guess.

PS: Just look at the first ten thousand bytes of this file. This is usually enough for a good guess about what the encoding is and is much faster than trying to look at the whole file. (Especially with a large file this can be very slow.) 

In [ ]:
# look at the first ten thousand bytes to guess the character encoding
import chardet
with open("../input/kickstarter-projects/ks-projects-201801.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

In [ ]:
# read in the file with the encoding detected by chardet
kickstarter_2016 = pd.read_csv("../input/kickstarter-projects/ks-projects-201612.csv", encoding='Windows-1252')

What if the encoding chardet guesses isn't right? Since chardet is basically just a fancy guesser, sometimes it will guess the wrong encoding. One thing you can try is looking at more or less of the file and seeing if you get a different result and then try that.

Set the parameter for rawdata.read(10000) to something higher/lower.

## Source: Scraping

In [ ]:
from bs4 import BeautifulSoup
import os
import glob
import pandas as pd
import pandas as pd

### Using os.listdir()
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'rt_html'

for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        # Your code here
        # Note: a correct implementation may take ~15 seconds to run
				# Parse and Clean data from webpage
        soup = BeautifulSoup(file, 'lxml')
        title = soup.find('title').contents[0][:-len(" - Rotten Tomatoes")]
        audience_score = soup.find('div', class_='audience-score').find('span', class_='superPageFontColor').contents[0][:-1]
        num_audience_ratings = soup.find('div', class_='audience-info').find_all('div')[1].contents[2].strip().replace(',','')
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

### Using glob.glob()
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
for ebert_review in glob.glob('ebert_reviews/*.txt'):
    with open(ebert_review, encoding='utf-8') as file:
        title = file.readline()[:-1]
        review_url = file.readline()[:-1]
        review_text = file.read()[:-1]
        
        break:

        # Append to list of dictionaries
        df_list.append({'title': title,
                        'review_url': review_url,
                        'review_text': review_text})
df = pd.DataFrame(df_list, columns = ['title', 'review_url', 'review_text'])

## Source: Downloading Files From The Internet

In [ ]:
# response.contents is in bytes format, not text, therefore we need to open the file in 'wb' (write binary) mode.

import requests
import os
# Make directory if it doesn't already exist
folder_name = 'ebert_reviews'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

ebert_review_urls = ['https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9900_1-the-wizard-of-oz-1939-film/1-the-wizard-of-oz-1939-film.txt',
                     'https://d17h27t6h515a5.cloudfront.net/topher/2017/September/59ad9901_2-citizen-kane/2-citizen-kane.txt']

# Implement the code in the video above in a for loop for all Ebert reviews
for url in ebert_review_urls:
    response = requests.get(url)
    # print (response.content)
    with open(os.path.join(folder_name,url.split('/')[-1]), mode="wb") as file:
        file.write(response.content)

## Source: Mashup of sources

If collecting data from multiple sources, ensure there's a column with unique values that you can use to merge between 2 sources. With multiple sources, these bridging unique values can be different columns for different sets of files being merged.

Loops through the Wikipedia page titles in *title_list* and:

- ***Stores the ranking of that movie in the Top 100 list based on its position in title_list. Ranking is needed so we can join this DataFrame with the master DataFrame later*.** We can't join on title because the titles of the Rotten Tomatoes pages and the Wikipedia pages differ.
- Uses **`[try` and `except` blocks](http://www.pythonforbeginners.com/error-handling/python-try-and-except)** to attempt to query MediaWiki for a movie poster image URL and to attempt to download that image. If the attempt fails and an error is encountered, the offending movie is documented in *image_errors*.
- Appends a dictionary with *ranking*, *title*, and *poster_url* as the keys and the extracted values for each as the values to *df_list*.

In [ ]:
import pandas as pd
import wptools
import os
import requests
from PIL import Image
from io import BytesIO

title_list = [
 'The_Wizard_of_Oz_(1939_film)',
 'Citizen_Kane',
 'The_Third_Man',
 'Get_Out_(film)',
 'Mad_Max:_Fury_Road',
 'The_Cabinet_of_Dr._Caligari',
 'All_About_Eve',
 'Inside_Out_(2015_film)',
 'The_Godfather',
 'Metropolis_(1927_film)',
 'E.T._the_Extra-Terrestrial',
 'Modern_Times_(film)']

folder_name = 'bestofrt_posters'
# Make directory if it doesn't already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# List of dictionaries to build and convert to a DataFrame later
df_list = []
image_errors = {}
for title in title_list:
    try:
        # This cell is slow so print ranking to gauge time remaining
        ranking = title_list.index(title) + 1
        print(ranking)
        page = wptools.page(title, silent=True)
        # Your code here (three lines)
        images = page.get().data['image']
        # First image is usually the poster
        first_image_url = images[0]['url']
        r = requests.get(first_image_url)
        # Download movie poster image
        i = Image.open(BytesIO(r.content))
        image_file_format = first_image_url.split('.')[-1]
        i.save(folder_name + "/" + str(ranking) + "_" + title + '.' + image_file_format)
        # Append to list of dictionaries
        df_list.append({'ranking': int(ranking),
                        'title': title,
                        'poster_url': first_image_url})
    
    # Not best practice to catch all exceptions but fine for this short script
    except Exception as e:
        print(str(ranking) + "_" + title + ": " + str(e))
        image_errors[str(ranking) + "_" + title] = images

for key in image_errors.keys():
    print(key)

# Inspect unidentifiable images and download them individually
for rank_title, images in image_errors.items():
    if rank_title == '22_A_Hard_Day%27s_Night_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/4/47/A_Hard_Days_night_movieposter.jpg'
    if rank_title == '53_12_Angry_Men_(1957_film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/9/91/12_angry_men.jpg'
    if rank_title == '72_Rosemary%27s_Baby_(film)':
        url = 'https://upload.wikimedia.org/wikipedia/en/e/ef/Rosemarys_baby_poster.jpg'
    if rank_title == '93_Harry_Potter_and_the_Deathly_Hallows_–_Part_2':
        url = 'https://upload.wikimedia.org/wikipedia/en/d/df/Harry_Potter_and_the_Deathly_Hallows_%E2%80%93_Part_2.jpg'
    title = rank_title[3:]
    df_list.append({'ranking': int(title_list.index(title) + 1),
                    'title': title,
                    'poster_url': url})
    r = requests.get(url)
    # Download movie poster image
    i = Image.open(BytesIO(r.content))
    image_file_format = url.split('.')[-1]
    i.save(folder_name + "/" + rank_title + '.' + image_file_format)

# Create DataFrame from list of dictionaries
df = pd.DataFrame(df_list, columns = ['ranking', 'title', 'poster_url'])
df = df.sort_values('ranking').reset_index(drop=True)
df

## Storing Data

### Writing to CSV
Great for simple datasets.

Cons:

- Lack of standards
- Data redundancy
- Sharing data can be cumbersome
- Not great for large datasets (see *"When does small become large?"* in the Cornell link in *More Information*)

In [ ]:
# save our file (will be saved as UTF-8 by default!)
kickstarter_2016.to_csv("ks-projects-201801-utf8.csv")

### Write to pickle file

In [ ]:
df.to_pickle("adataset_clean.pkl")

### Save to Database
Pros:

Fast. Scales with data, can handle large amounts of data. Can be queried. Can access data where it lives, no need to make copies. Easy to audit and replicate the analysis. Can run queries on multiple tables. Can answer deeper questions than regular dashboards. Check for data integrity (for e.g. type of column type). Can be accessed by multiple people concurrently.

In [ ]:
df.to_sql('master', engine, index=False)
df_gather = pd.read_sql('SELECT * FROM master', engine)

In the context of data wrangling, it's recommended that databases and SQL only come into play for gathering data or storing data. That is:

- **Connecting to a database and importing data** into a pandas DataFrame (or the analogous data structure in your preferred programming language), then assessing and cleaning that data, or
- **Connecting to a database and storing data** you just gathered (which could potentially be from a database), assessed, and cleaned

These tasks are especially necessary when you have large amounts of data, which is where SQL and other databases excel over flat files.

The two scenarios above can be further broken down into three main tasks:

- Connecting to a database in Python
- Storing data ***from*** a pandas DataFrame ***in*** a database to which you're connected, and
- Importing data ***from*** a database to which you're connected ***to*** a pandas DataFrame

# EDA

**Exploring or EDA** will help you augment your data to maximize the potential of your analysis, viz and model. We use visualizations to summarize our data's main characteristics. From there, we remove outliers, create more descriptive features from existing ones (feature engineering.)

e.g.
- Using summary statistics like `count` on the state column or `mean` on the weight column to see if patients from certain states or of certain weights are more likely to have diabetes, which we can use to exclude certain patients from the analysis and make it less biased.

* Create a data dictionary with the column name, data type, range of values and notes on each column

EDA usually involves a combination of the following methods:

* **Univariate** visualization of and summary statistics for each field in the raw dataset, specifically for variables of interest
* **Bivariate** visualization and summary statistics for assessing the relationship between each variable in the dataset and the target variable of interest (e.g. time until churn, spend)
* **Multivariate** visualizations to understand interactions between different fields in the data
* Dimensionality reduction to understand the fields in the data that account for the most variance between observations and allow for the processing of a reduced volume of data
* Clustering of similar observations in the dataset into differentiated groupings, which by collapsing the data into a few small data points, patterns of behavior can be more easily identified

**More data usually results in better accuracy than a finely trained model.**

### How many observations are there?

In [ ]:
df.shape[0]

### How many features?

In [ ]:
df.shape[1]

### Example observations
Then, you'll want to display example observations from the dataset. This will give you a "feel" for the values of each feature.

Thank about:
Do the columns make sense?
Do the values in those columns make sense?
Are the values on the right scale?
Is missing data going to be a big problem based on a quick eyeball test?

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.sample(5)

In [ ]:
df.info()

### What does a datapoint of interest look like?

What values across different features might it have?

Make a list of them.

How many POI datapoints exist?

What information do you need to google to better inform your intuition around POIs.

### What are some interesting questions that you want to ask of this data?

Make a list of them.

Questions about POIs, total datapoints that have a certain value for a feature and how many of these are POIs, what values do certain features take on, how are missing values denoted, what % of datapoints have values for interesting features? Are Nans/0s associated with one type of categorical value for the label?

## Univariate Analysis
* Look at one variable at a time.
* Summarize and examize

Distribution of a variable:
* What values the variable takes
* How often the variable takes those values


#### Data Types in Pandas
<img src="pandas_dtypes.png">

### Categorical variables
* There is less available options with categorical variables
* Count the frequency of each variable
* Low frequency strings might be outliers
* You might want to relabel low frequency strings 'other'
* Find the number of unique labels for each column
* In pandas, change the data type to categorical (better when there aren't too many unique values)
* Bar plots of counts
* String columns allow for feature engineering by splitting the string, counting certain letters, finding the length of, etc... Feature engineering can be done later when modeling
* See if a numerical/datetime variable has a string type incorrectly because there is an 'NA'/'missing' value or '%/$' other sign

A note on df.describe()
* df.describe(include='all') - Describing all columns of a DataFrame regardless of data type.
* df.describe(include=[np.number]) - Including only numeric columns
* df.describe(include=[np.object]) - Including only string columns
* df.describe(include=['category']) - Including only categorical columns

In [ ]:
# Select all features on type categorical/string
df.select_dtypes(include=['category'])
df.select_dtypes(include=['object'])

### Categorical variables

* .count() - gives number of total values in column
* .unique() - returns array of all unique values in that column
* .value_counts() - returns object containing counts of unique values

In [ ]:
# If an int value is being treated as a string value by pandas,
# use this to convert the variable to numeric before calling the value_counts function
# Do this for each column you run value_counts on
df[col_name] = pandas.to_numeric(df[col_name], errors='coerce')

In [ ]:
df.describe(include=['category'])
df.describe(include=['object'])

#### One Way Frequency Table


<img src="wrangling-notebook-1.png">

<img src="wrangling-notebook-2.png">

In [ ]:
# Code to combine both the above cells and get freq table and proportions in one table
freq_table = pd.crosstab(index=df["payer_name"],  # Make a crosstab
                              columns="count")      # Name the count column
freq_table_per = pd.crosstab(index=df["payer_name"],  # Make a crosstab
                              columns="percentage", normalize=True)
freq_table['percentage'] = freq_table_per['percentage']
freq_table.sort_values(by='count', ascending=False)

In [ ]:
# TODO: Add bar chart

##### Code to do the above in an uncompressed way. Can ignore this and move to 2-way frequency table:

In [ ]:
# One Way Frequency Table
freq_table = pd.crosstab(index=df["Symbol"],  # Make a crosstab
                              columns="count")      # Name the count column
freq_table

In [ ]:
# Proportions of each value

# Method 1
freq_table = pd.crosstab(index=df["payer_name"],  # Make a crosstab
                              columns="count", normalize=True)
# value_counts() excludes NA values, so numbers might not add up to 1 if there are missing values. This is okay because we care about the % of each value to take the missing values in account.

# Method 2
prop = freq_table / freq_table.sum()
prop = prop.sort_values('count', ascending=False)

# Show top 10 values (by proportion) in column
prop[:10]

In [ ]:
# How many times a value occurs
# Method 1
df.col_name.value_counts(dropna=False)
# Method 2
df.groupby(col_name).size()

# The percentage of how much the value occurs
# Method 1
df.col_name.value_counts(dropna=False, normalize=True)
# Method 2
df.groupby(col_name).size() * 100 / len(df)

#### Two Way Frequency Table

<img src="wrangling-notebook-3.png">

<img src="wrangling-notebook-4.png">

<img src="wrangling-notebook-5.png">

#### Higher Dimensional Frequency Table

<img src="wrangling-notebook-6.png">

In [ ]:
# Higher Dimensional Frequency Table
surv_s_class = pd.crosstab(index=titanic_train["Survived"], 
                             columns=[titanic_train["Pclass"],
                                      titanic_train["Sex"]],
                             margins=True)
surv_s_class

In [ ]:
# Proportions of each value
prop = surv_s_class/surv_sex_class.loc["All"]    # Divide by column totals

prop = prop.sort_values('count', ascending=False)

# Show top 10 values (by proportion) in column
prop[:10]

In [ ]:
# TODO: Add stacked bar chart

### Numerical variables
* There are a lot more options for continuous variables
* Use the five number summary - with **`.describe`**
* Boxplots are great ways to find outliers
* Use histograms and kernel density estimators to visualize the distribution.
* Know the shape of the distribution
* Think about making categorical variables out of continuous variables by cutting them into bins.

In [ ]:
# Select all features on type numerical
df.select_dtypes(include=['number'])

In [ ]:
# Do the min, max values make sense? Do the mean, std and other values seem what you would expect or does anything stand out about them?
df.describe()

In [ ]:
# Create a dist plot (also a box and whiskers plot)
hist_kws={"alpha": 0.3}
sns.distplot(df[col_name], hist_kws=hist_kws)

In [ ]:
ax = sns.barplot(x="x", y="x", data=df, estimator=lambda x: len(x) / len(df) * 100)
ax.set(ylabel="Percent")

In [ ]:
df.col_name.value_counts()

In [ ]:
# Various methods of indexing and selecting data (.loc and bracket notation with/without boolean indexing, also .iloc)
df.loc[df['city'] == "New York"]


#### Use bootstrapping to get more 'samples'
* Bootstrapping is done by resampling your data with replacement and gives you a 'new' random dataset
* This helps you get multiple looks at the data
* You can get estimates for the mean and variance of continuous columns this way.


### Missing Values


#### A note on missing values and different sources of data:

Adding in the new POI’s in this example, none of whom we have financial information for, has introduced a subtle problem, that our lack of financial information about them can be picked up by an algorithm as a clue that they’re POIs. Another way to think about this is that there’s now a difference in how we generated the data for our two classes--non-POIs all come from the financial spreadsheet, while many POIs get added in by hand afterwards. That difference can trick us into thinking we have better performance than we do--suppose you use your POI detector to decide whether a new, unseen person is a POI, and that person isn’t on the spreadsheet. Then all their financial data would contain “NaN” but the person is very likely not a POI (there are many more non-POIs than POIs in the world, and even at Enron)--you’d be likely to accidentally identify them as a POI, though!

This goes to say that, when generating or augmenting a dataset, you should be exceptionally careful if your data are coming from different sources for different classes. It can easily lead to the type of bias or mistake that we showed here. There are ways to deal with this, for example, you wouldn’t have to worry about this problem if you used only email data--in that case, discrepancies in the financial data wouldn’t matter because financial features aren’t being used. There are also more sophisticated ways of estimating how much of an effect these biases can have on your final answer; those are beyond the scope of this course.

For now, the takeaway message is to be very careful about introducing features that come from different sources depending on the class! It’s a classic way to accidentally introduce biases and mistakes.

#### Calculating missing values

In [ ]:
# All columns
missing_values_count = df.isnull().sum()
missing_values_count

In [ ]:
# how many total missing values do we have in the dataframe?
total_cells = np.product(df.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing in the dataframe
(total_missing/total_cells) * 100

In [ ]:
df[df['col_name'].isnull()]

In [ ]:
# How many missing values exist. Missing values may exist as dashes, slases, zeroes, NA, or none. Account for all of these. Also 0 and null are different because they lead to diff values of std, variance etc.)
sum(df.col_name.isnull())

#### Plot missing values

In [ ]:
# The 1st argument contains the x-values (column names), the 2nd argument the y-values (missing counts).
base_color = sb.color_palette()[0]
sb.barplot(missing_values_count.index.values, missing_values_count, color = base_color)

### Outliers
* Use your natural human ability to look at boxplots to find thresholds for what an outlier might be
* Generate a new column of data that is 0/1 for outlier or not. This will quickly help you find them later.


Outliers can cause problems with certain types of models. For example, linear regression models are less robust to outliers than decision tree models.

In general, if you have a legitimate reason to remove an outlier, it will help your model’s performance.

However, **outliers are innocent until proven guilty**. You should never remove an outlier just because it’s a "big number." That big number could be very informative for your model.

You must have a good reason for removing an outlier, such as suspicious measurements that are unlikely to be real data.

One option is to try a transformation. Square root and log transformations both pull in high numbers. This can make assumptions work better if the outlier is a dependent variable and can reduce the impact of a single point if the outlier is an independent variable.

**IQR Method:**
A commonly used rule says that a data point is an outlier if it is more than 1.5 . IQR above the third quartile or below the first quartile.
Said differently, low outliers are < Q1 - (1.5 . IQR) and high outliers are > Q3 + (1.5 . IQR)

** Z-score Method **

The intuition behind the Z-score method of outlier detection is that, once we’ve centred and rescaled the data, anything that is too far from zero (the threshold is usually a Z-score of 3 or -3) should be considered an outlier.

The Z-score method relies on the mean and standard deviation of a group of data to measure central tendency and dispersion. This is troublesome, because the mean and standard deviation are highly affected by outliers – they are not robust. In fact, the skewing that outliers bring is one of the biggest reasons for finding and removing outliers from a dataset!

So we use the modified Z-score method is that it uses the median and MAD rather than the mean and standard deviation. The median and MAD are robust measures of central tendency and dispersion, respectively.

In [ ]:
df.col_name.sort_values()
# See if the min or max values are too far away from the avg value (this could be inaccurate or this could be because of a difference in units: one value in KGs while the others are in LBs)

In [ ]:
# get all numerical columns
df_num = df.select_dtypes(include=[np.number]).columns.tolist()
df_num

# Calculate modified z_score
for col in df_num:
    df_outliers = df[(np.abs(df[col]-df[col].median()) > (3.5*df[col].mad()))]
    
df_outliers

Another robust measure of dispersion.

PS: One caveat: both of these methods will encounter problems with a strongly skewed dataset. If the data is distributed in a strongly asymmetrical fashion, it will need to be re-expressed before applying any of these methods.

In [ ]:
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))
outliers = outliers_iqr(df.dock_count)
df.loc[outliers]

In [ ]:
# ^ the 2 methods above (modified z_score and outliers_iqr) seem to work the best, but try all methods to see what works for for different datasets

**Others Ways:**

In [ ]:
def outliers_modified_z_score(ys):
    threshold = 3.5

    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in ys]
    return np.where(np.abs(modified_z_scores) > threshold)

In [ ]:
# See if the min or max values are too far away from the avg value (this could be inaccurate or this could be because of a difference in units: one value in KGs while the others are in LBs)
df.col_name.sort_values()
df.col_name.max()
df.col_name.min()

# Fetch outliers
very_large_value = 100000000
df[df.col_name > very_large_value]

#### Plot Outliers

In [ ]:
# you might need to change the limits or scale of the axis to take a closer look at the outliers in the data
plt.xlim(0, 40)

### Upper/Lower bounded values

Sometimes values are cut off at an arbitrary high or low value and therefore, the first and last value have a lot of datapoints. Check for this. This is very important to remember when assessing the generalizability of models trained on this later. Might want to mention this as part of dataset insights.


### Duplicated data
* Lots of data gets accidentally duplicated. Check for duplicates or near duplicates of rows and columns
* If any columns are calculated entirely by that of another column or columns (like with depth from the diamonds data), ensure the calculation holds. 

Duplicate observations most frequently arise during data collection, such as when you:
* Combine datasets from multiple places
* Scrape data

In [ ]:
# See if there's any duplicate values(cause of nicknames/misspellings/multiple ways of expressing a value), or default 'John Doe' type values
df[df.col_name.duplicated()]

In [ ]:
# Duplicated columns
# See if there are a lot of duplicated columns across tables which means that the tables could be combined. Ideally only the id column(s) should be common across tables
all_columns = pd.Series(list(first) + list(second) + list(third))
all_columns[all_columns.duplicated()]

### Imbalanced Classes

In [ ]:
# TODO: Add how to identify Imabalanced classes

### Irrelevant data

Irrelevant observations are those that don’t actually fit the specific problem that you’re trying to solve.

For example, if you were building a model for Single-Family homes only, you wouldn't want observations for Apartments in there.
This is also a great time to review your charts from Exploratory Analysis. You can look at the distribution charts for categorical features to see if there are any classes that shouldn’t be there.
Checking for irrelevant observations before engineering features can save you many headaches down the road.

### Plots

#### Summary

| Univariate             | Graphical                               | Non-Graphical                     | 
|-------------|-----------------------------------------|-----------------------------------|
| Categorical | Bar char of frequencies (count / percent) | Frequency/Contingency table (count / percent) |
| Num/Continuous  | Histogram/rugplot/KDE, box/violin/swarm, qqplot, fat tails  | central tendency -mean/median/mode, spread - variance, std, skew, kurt, IQR  |

| Bivariate/multivariate            | Graphical                               | Non-Graphical                     | 
|-------------|-----------------------------------------|-----------------------------------|
| Categorical vs Categorical | heat map, mosaic plot | Two-way Contingency table (count/percent) |
| Continuous vs Continuous  | all pairwise scatterplots, kde, heatmaps |  all pairwise correlation/regression   |
| Categorical vs Continuous  | [bar, violin, swarm, point, strip seaborn plots](http://seaborn.pydata.org/tutorial/categorical.html)  | Summary statistics for each level |

#### Figures, Axes, Subplots, Legends

In [ ]:
# set figure size (width, height)
plt.figure(figsize = [10, 5])

# Create subplots
# creates a new Axes in our Figure, its size determined by the subplot() function arguments. The first two arguments says to divide the figure into one row and two columns, and the third argument says to create a new Axes in the first slot. Slots are numbered from left to right in rows from top to bottom. Note in particular that the index numbers start at 1 (rather than the usual Python indexing starting from 0).
# subplot 1: example of somewhat too-large bin size
plt.subplot(1, 2, 1) # 1 row, 2 cols, subplot 1
bin_edges = np.arange(0, df['num_var'].max()+4, 4)
plt.hist(data = df, x = 'num_var', bins = bin_edges)

# subplot 2: example of somewhat too-small bin size
plt.subplot(1, 2, 2) # 1 row, 2 cols, subplot 2
bin_edges = np.arange(0, df['num_var'].max()+1/4, 1/4)
plt.hist(data = df, x = 'num_var', bins = bin_edges)

# Creating a bunch of subplots
fig, axes = plt.subplots(3, 4) # grid of 3x4 subplots
axes = axes.flatten() # reshape from 3x4 array into 12-element vector
for i in range(12):
    plt.sca(axes[i]) # set the current Axes
    # do something with the current axis.. in this case print the index number in the middle of the axis
    plt.text(0.5, 0.5, i+1)

# Move legend outside the chart
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

<img src='wrangling-notebook-22.png'>

#### Categorical Variable Plots

In [ ]:
# Retain the order of categorical variable values in charts:
# Specify order in cat_classes and run this code
cat_classes = ['Minicompact Cars', 'Subcompact Cars', 'Compact Cars', 'Midsize Cars', 'Large Cars']
pd_ver = pd.__version__.split(".")
if (int(pd_ver[0]) > 0) or (int(pd_ver[1]) >= 21): # v0.21 or later
    vclasses = pd.api.types.CategoricalDtype(ordered = True, categories = cat_classes)
    fuel_econ['VClass'] = fuel_econ['VClass'].astype(vclasses)
else: # pre-v0.21
    fuel_econ['VClass'] = fuel_econ['VClass'].astype('category', ordered = True,
                                                     categories = cat_classes)

Caterogical variables can be visualized using bar plots.

In particular, you'll want to look out for sparse classes, which are classes that have a very small number of observations.

A "class" is simply a unique value for a categorical feature.CAterogical variables can be visualizing

Classes with short bars are sparse classes. They tend to be problematic when building models.

In the best case, they don't influence the model much.
In the worse case, they can cause the model to be overfit.
Therefore, make a note to combine or reassign some of these classes later.

<img src="https://34tzkp3af7ck1k675g1stf6n-wpengine.netdna-ssl.com/wp-content/uploads/2017/06/grouping-sparse-classes-before.png">

In [ ]:
df[col_name].value_counts(normalize=True).plot(kind='barh')

In [ ]:
sns.countplot(x='col_name', data=df)

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,6))
df['col_name'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=False)
ax[0].set_title('col_name')
ax[0].set_ylabel('')
sns.countplot('col_name',data=df,ax=ax[1])
ax[1].set_title('col_name')
plt.show()

#### Bar Charts (Categorical)

In [ ]:
# For nominal data:
# arrange bar chart in order of frequency. Don't rearrange for ordinal data,
# the inherent order or the levelswill be more important to convey
# Start your chart with 0 as the baseline to avoid information distortion
# If you have a lot of categories, or the category names are long, a horizontal orientation might be more convinient

In [ ]:
# start by plotting everything in one color, and sort by frequency
base_color = sb.color_palette()[0]
cat_order = df['cat_var'].value_counts().index
sns.countplot(data = df, x = 'cat_var', color = base_color, order = cat_order)

In [ ]:
# For ordinal data:
# By default, pandas reads in string data as object types, and will plot the bars in the order
# in which the unique values were seen. By converting the data into an ordered type, the order
# of categories becomes innate to the feature, and we won't need to specify an "order" parameter
# each time it's required in a plot.

# convert variable to ordered type
level_order = ['Alpha', 'Beta', 'Gamma', 'Delta']
ordered_cat = pd.api.types.CategoricalDtype(ordered = True, categories = level_order)
df['cat_var'] = df['cat_var'].astype(ordered_cat)

# plot the variable
base_color = sb.color_palette()[0]
sb.countplot(data = df, x = 'cat_var', color = base_color)

In [ ]:
# rotate category variables
plt.xticks(rotation = 90)

In [ ]:
# horizontal bar chart - change x parameter to y
sns.countplot(data = df, y = 'cat_var', color = base_color, order = cat_order)

In [ ]:
# Relative frequency - Plotting proportions on the axis instead of absolute counts

# get proportion taken by most common group for derivation
# of tick marks
n_points = df.shape[0]
max_count = df['cat_var'].value_counts().max()
max_prop = max_count / n_points

# generate tick mark locations and names
tick_props = np.arange(0, max_prop, 0.05)
tick_names = ['{:0.2f}'.format(v) for v in tick_props]

# create the plot
base_color = sb.color_palette()[0]
sb.countplot(data = df, x = 'cat_var', color = base_color)
# plt.yticks(tick_position, tick_labels)
plt.yticks(tick_props * n_points, tick_names)
plt.ylabel('proportion')

<img src='wrangling-notebook-20.png'>

In [ ]:
# Relative frequency variation - Plotting absolute counts on axis and porportions on the bars

# create the plot
base_color = sb.color_palette()[0]
sb.countplot(data = df, x = 'cat_var', color = base_color)

# add annotations
n_points = df.shape[0]
cat_counts = df['cat_var'].value_counts()
locs, labels = plt.xticks() # get the current tick locations and labels

# loop through each pair of locations and labels
for loc, label in zip(locs, labels):

    # get the text property for the label to get the correct count
    count = cat_counts[label.get_text()]
    pct_string = '{:0.1f}%'.format(100*count/n_points)

    # print the annotation just below the top of the bar
    plt.text(loc, count-8, pct_string, ha = 'center', color = 'w')

<img src='wrangling-notebook-21.png'>

In [ ]:
# barplot() - if data is summarized and you still want to build a bar chart.
# countplot() - if data is not yet summarized, so that you don't need to do extra summarization work

#### Pie Chart / Donut Plot (Categorical)

In [ ]:
# Bar chart is better than pie/donut charts aas its hard to get an accurate estimate of the relative
# frequency using just the pie/donut chart alone. If there are a lot of categories, you have to choose
# a few to plot. If you have a lot of categories, or categories that have small proportions,
# consider grouping them together so that fewer wedges are plotted, or use an 'Other' category.

# If these guidelines cannot be met, use a bar chart instead. A bar chart is a safer choice in general.
# The bar heights are more precisely interpreted than areas or angles, and a bar chart can be displayed
# more compactly than a pie chart. There's also more flexibility with a bar chart for plotting variables
# with a lot of levels, like plotting the bars horizontally.

# User these charts when you want to show how the data is broken down into parts
# and only plot 2-3 categories so the relative portions are distinct
# Start at the top and plot clockwise according to frequency

# Pie Chart
# The axis function call and 'square' argument makes it so that the scaling of the plot is equal on both the x- and y-axes. Without this call, the pie could end up looking oval-shaped, rather than a circle.
sorted_counts = df['cat_var'].value_counts()
plt.pie(sorted_counts, labels = sorted_counts.index, startangle = 90,
        counterclock = False);
plt.axis('square')

In [ ]:
# Donut Chart
sorted_counts = df['cat_var'].value_counts()
plt.pie(sorted_counts, labels = sorted_counts.index, startangle = 90,
        counterclock = False, wedgeprops = {'width' : 0.4});
plt.axis('square')

#### Waffle Plot (Categorical)

In [ ]:
# There's no built-in function for waffle plots in Matplotlib or Seaborn, so we'll need to take some additional steps in order to build one with the tools available.
# The blocks are drawn from left to right, bottom to top
# First, we need to create a function to decide how many blocks to allocate to each category. The function below, percentage_blocks, uses a rule where each category gets a number of blocks equal to the number of full percentage points it covers. The remaining blocks to get to the full one hundred are assigned to the categories with the largest fractional parts.
def percentage_blocks(df, var):
    """
    Take as input a dataframe and variable, and return a Pandas series with
    approximate percentage values for filling out a waffle plot.
    """
    # compute base quotas
    percentages = 100 * df[var].value_counts() / df.shape[0]
    counts = np.floor(percentages).astype(int) # integer part = minimum quota
    decimal = (percentages - counts).sort_values(ascending = False)
    # add in additional counts to reach 100
    rem = 100 - counts.sum()
    for cat in decimal.index[:rem]:
        counts[cat] += 1
    return counts

# Second, plot those counts as boxes in the waffle plot form
waffle_counts = percentage_blocks(df, 'cat_var')
prev_count = 0
# for each category,
for cat in range(waffle_counts.shape[0]):
    # get the block indices
    blocks = np.arange(prev_count, prev_count + waffle_counts[cat])
    # and put a block at each index's location
    x = blocks % 10 # use mod operation to get ones digit
    y = blocks // 10 # use floor division to get tens digit
    plt.bar(x = x, height = 0.8, width = 0.8, bottom = y)
    prev_count += waffle_counts[cat]

# Third, we need to do involve aesthetic cleaning to polish it up for interpretability. We can take away the plot border and ticks, since they're arbitrary, but we should change the limits so that the boxes are square. We should also add a legend so that the mapping from colors to category levels is clear.
# aesthetic wrangling
plt.legend(waffle_counts.index, bbox_to_anchor = (1, 0.5), loc = 6)
plt.axis('off')
plt.axis('square')

<img src='wrangling-notebook-25.png'>

In [ ]:
# Other variants of the waffle plot exist to extend it beyond just displaying probabilities.
# By associating each square with an amount rather than a percentage, we can use waffle plots to show absolute frequencies instead. This might cause us to end up with something other than 100 squares.
# each box represents five full counts
# boxes are plotted in columns from left to right
num_counts = 5
waffle_counts = (df['cat_var'].value_counts() / num_counts).astype(int)

prev_count = 0
# for each category,
for cat in range(waffle_counts.shape[0]):
    # get the block indices
    blocks = np.arange(prev_count, prev_count + waffle_counts[cat])
    # and put a block at each index's location
    x = blocks % 10
    y = blocks // 10
    plt.bar(y, 0.8, 0.8, x)
    prev_count += waffle_counts[cat]

# box size legend
plt.bar(7.5, 0.8, 0.8, 2, color = 'white', edgecolor = 'black', lw = 2)
plt.text(8.1, 2.4,'= 5 data points', va = 'center')

# aesthetic wrangling
plt.legend(waffle_counts.index, bbox_to_anchor = (0.8, 0.5), loc = 6)
plt.axis('off')
plt.axis('square')

# Notes:
# PyWaffle makes it easier to create Waffle Plots: https://github.com/ligyxy/PyWaffle
# Using icons for each tally, rather than just squares - Infographics often take this approach, by having each icon represent some number of units (e.g. one person icon representing one million people). But while it can be tempting to use icons to represent values as a bit of visual flair, an icon-based plot contains more chart junk than a bar chart that conveys the same information. There’s a larger cognitive challenge in having to count a number of icons to understand the scale of a value, compared to just referencing a box's endpoint on a labeled axis.
# The effort required to create a meaningful and useful waffle plot means that it is best employed carefully as a part of explanatory visualizations. During the exploratory phase, you're better off using more traditional plots like the bar chart to more rapidly build your understanding of the data.

<img src='wrangling-notebook-26.png'>

#### Numerical Variable Plots

Often, a quick and dirty grid of histograms is enough to understand the distributions

Here are a few things to look out for:

- Distributions that are unexpected
- Potential outliers that don't make sense
- Features that should be binary (i.e. "wannabe indicator variables")
- Boundaries that don't make sense
- Potential measurement errors

PS: When there are extremely large differences between the min and max values, and the plot will need to be adjusted accordingly. In such cases, it's good to look at the plot on a **log scale**. The keyword arguments logx=True or logy=True can be passed in to .plot() depending on which axis you want to rescale.

<img src="https://34tzkp3af7ck1k675g1stf6n-wpengine.netdna-ssl.com/wp-content/uploads/2017/06/histogram-grid-example.png">

In [ ]:
# distribution charts for all independent, numerical variables together
# assuming independent variables start at index 2
for column in df.columns[2:]:
    print(column)
    #s=df['column']
    s=df[column]
    mu, sigma =norm.fit(s)
    count, bins, ignored = plt.hist(s, 30, normed=True, color='g')
    plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (bins - mu)**2 / (2 * sigma**2) ), linewidth=1, color='r')

    title = "Plot used: mu = %.2f,  std = %.2f" % (mu, sigma)
    plt.title(title, loc='right')

    plt.show()

In [ ]:
# KDE Plot
sns.kdeplot(df.col_name)
plt.show()

# KDE + Hist
hist_kws={"alpha": 0.3}
sns.distplot(df['col_name'], hist_kws=hist_kws)

# Optional: Trim long-tail/other values and create KDE again
df_trimmed = df.loc[df.index[df.price < ceiling_value]].sample(n=50000)
sns.kdeplot(df_trimmed.col_name)
plt.show()

Some of these variables may not be close to a normal distribution, but that might not be very critical if the techniques we apply are not very sensitive to normality.
There's also the option of converting some of these variables to categories for model training.

#### Histogram (Numerical)

In [ ]:
# specify bin values explicitly
bin_edges = np.arange(0, df['num_var'].max()+1, 1)
# larger bin size
bin_edges = np.arange(0, df['num_var'].max()+5, 5)
plt.hist(data = df, x = 'num_var', bins = bin_edges)

# for discrete data,
# rwidth =0.7 adds space between bars to emphasize discrete values, cause the bars will take up 70% of the space allocated by each bin, with 30% of the space left empty
plt.hist(data = df, x = 'num_var', bins = bin_edges, rwidth = 0.7)

In [ ]:
# Change Axis Limits
# Note any aspects of the data like number of modes and skew, and note the presence of outliers in the data for further investigation.
# you might need to change the limits or scale of the axis to take a closer look at the underlying patterns in the data
# use of axis limits can allow focusing on data points in that range without needing to go through creation of a new DataFrame filtering out the data points in the latter group (> 40)
plt.xlim(0, 40)

In [ ]:
# Scaling the graph
# Logarithmic scaling captures the multiplicative differences rather than the arithmetic differences
# Each tick mark is twice the value of the previous bar
# Lognormal distribution: The data has a normal distribution after a log transform - i.e. data that, in its natural units, can look highly skewed: e.g. lots of points with low values, with a very long tail of data points with large values.
# (this suggests that building a logarithmic model for the variable might work better than a linear model - linear changes in predictor features will result in multiplicative effects in y)
# all values must be +ve in order to use a log transform
# For visualizations it is best to do axis transformations rather than data transformations
# We can transform the data during the analysis phase
# Here we aren't changing the values taken by the data, only how they're displayed
# Between integer powers of 10, we don't have clean values for even markings, but we can still get close. Setting ticks in cycles of 1-3-10 or 1-2-5-10 are very useful for base-10 log transforms.
plt.figure(figsize = [10, 5])

# left histogram: data plotted in natural units
plt.subplot(1, 2, 1)
bin_edges = np.arange(0, data.max()+100, 100)
plt.hist(data, bins = bin_edges)
plt.xlabel('values')

# right histogram: data plotted after direct log transformation
bin_edges = 10 ** np.arange(0.8, np.log10(data.max())+0.1, 0.1)
plt.hist(data, bins = bin_edges)
plt.xscale('log')
tick_locs = [10, 30, 100, 300, 1000, 3000]
plt.xticks(tick_locs, tick_locs)

<img src='wrangling-notebook-23.png' width=50%>
<img src='wrangling-notebook-24.png'>

#### Distplot (Numerical)

In [ ]:
# if you want a quick start on choosing a representative bin size for histogram plotting, you might take a quick look at the basic distplot first before getting into the customization
# distplot function has built-in rules for specifying histogram bins, and by default plots a curve depicting the kernel density estimate (KDE) on top of the data. The vertical axis is based on the KDE, rather than the histogram: you shouldn't expect the total heights of the bars to equal 1, but the area under the curve should equal 1
bin_edges = np.arange(0, df['num_var'].max()+1, 1)
sb.distplot(df['num_var'], hist_kws = {'alpha' : 0.8})
# Despite the fact that the default bin-selection formula used by distplot might be better than the choice of ten bins that .hist uses, you'll still want to do some tweaking to align the bins to 'round' values.
sb.distplot(df['num_var'], bins = bin_edges, hist_kws = {'alpha' : 0.8})

#### Box and whiskers (Numerical)

In [ ]:
base_color = sns.color_palette()[0]
sns.boxplot(data = df, x = 'cat_var', y = 'num_var', color = base_color)
# switch x and y parameter values to make this a horizontal chart

#### Violin Plot (Visualize the distribution of one numerical variable or lots of categorical variables)

In [ ]:
# Area depicts the distribution of points. > width = > the number of points
base_color = sns.color_palette()[0]
sns.violinplot(data = df, x = 'cat_var', y = 'num_var', color = base_color)
# switch x and y parameter values to make this a horizontal chart

#### Segmentations - Categorical + Numerical Variables Together

Segmentations are powerful ways to observe the relationship between categorical features and numeric features.

Box plots allow you to do so.

Here are a few insights you could draw from the following chart.

The median transaction price (middle vertical bar in the box) for Single-Family homes was much higher than that for Apartments / Condos / Townhomes.
The min and max transaction prices are comparable between the two classes.

<img src="https://34tzkp3af7ck1k675g1stf6n-wpengine.netdna-ssl.com/wp-content/uploads/2017/06/boxplot-segmentation-example.png">

### Types of distributions

Described their their:
+ Shape
+ Center
+ Spread

**Shape:**
<img src="wrangling-notebook-7.png">
          
All of these are symmetric, but differ in their peakness or modality (unimodal, bimodal, uniform).

Modes - values that occur most often.
The mode doesn't always occur at the center.
The center is the midpoint, the value that divides the distribution so that half of the values take smaller values and the other half take larger values.
Spread - approx range covered by the distribution. Measured by range.

Distributions can be both left/right kewed and have more than one mode.
<img src="wrangling-notebook-8.png">

The histogram can give us the shape, but the center and spread are harder to determine from the histrogram.

**Center:**
* Mean: Average = sum of values/n
* Median: Write the values in ascending order. Median is the middle value or the avg of the middle 2 values.
* Mode: Value with the highest frequency

<img src="wrangling-notebook-9.png">
These distributions have the same center. So spread alone isn't enough to describe a distribution. We also need to talk about the spreads which are different for each distribution above.

**Spread:**
* **Standard Deviation** - Typical/avg difference between the observations and the mean. Sqrt(variance). We take the sqrt inorder to bring the units back to units (we squared them while calculating variance)
* **Variance** - |(x - x^)^2|/(n-1)

## Bivariate and Multivariate EDA

Under bivariate analysis we will examine the realtionship between Dependent variable and each of the independent variables. We will also check selected pairs of independent variables.

Boxplots are great when you have a numeric column that you want to compare across different categories.
When you want to visualize two numeric columns, scatter plots are ideal.

### Categorical vs Categorical
* Create two way contingency table of frequency counts
* Create a heat map
* Find expected counts and possibly do a chi-squared test

### Categorical vs Continuous
* Use the seaborn categorical plots

### Continuous vs Continuous
* Plot all combinations of scatterplots
* Use a hierarchical clustering plot

### Correlations

In general, you should look out for:

Which features are strongly correlated with the target variable?
Are there interesting or unexpected strong correlations between other features?

Note:
Correlation is a value between -1 and 1 that represents how closely two features move in unison. You don't need to remember the math to calculate them. Just know the following intuition:

Positive correlation means that as one feature increases, the other increases. E.g. a child’s age and her height.
Negative correlation means that as one feature increases, the other decreases. E.g. hours spent studying and number of parties attended.
Correlations near -1 or 1 indicate a strong relationship.
Those closer to 0 indicate a weak relationship.
0 indicates no relationship.

In bivariate analysis, there's a dependent and an independent variable.

** X is a predictor of Y. **

<img src="wrangling-notebook-10.png">

Determine which variable plays which role in our analysis.

Use this classification system to determine which plots and statistical tools to use:

<img src="wrangling-notebook-12.png">

<img src="wrangling-notebook-11.png">

<img src="** Visualizing_Flowchart.png">

** Types of relationships shown in scatter plots **
<img src="wrangling-notebook-13.png">

<img src="wrangling-notebook-14.png">



In [ ]:
# Pearson correlation
df.corr()

# Kendall Tau correlation
df.corr('kendall')

# Spearman Rank correlation
df.corr('spearman')

Kendall and Spearman correlation measures require you to rank the data before calculating the coefficients. You can easily do this with rank().

Furthermore, there are some assumptions that these correlations work with: the Pearson correlation assumes that your variables are normally distributed, that there is a straight line relationship between each of the variables and that the data is normally distributed about the regression line. The Spearman correlation, on the other hand, assumes that you have two ordinal variables or two variables that are related in some way, but not linearly.

The Kendall Tau correlation is a coefficient that represents the degree of concordance between two columns of ranked data. You can use the Spearman correlation to measure the degree of association between two variables. These seem very similar to each other, don’t they?

Even though the Kendal and the Spearman correlation measures seem similar, but they do differ: the exact difference lies in the fact that the calculations are different. The Kendal Tau coefficient is calculated by the number of concordant pairs minus the number of discordant pairs divided by the total number of pairs. The Spearman coefficient is the sum of deviation squared by n times n minus 1.

Spearman’s coefficient will usually be larger than the Kendall’s Tau coefficient, but this is not always the case: you’ll get a smaller Spearman’s coefficient when the deviations are huge among the observations of your data. The Spearman correlation is very sensitive to this and this might come in handy in some cases!

So, when do you want to use which coefficient, because the two of these correlation actually test something different; Kendall’s Tau is representing the proportion of concordant pairs relative to discordant pairs and the Spearman’s coefficient doesn’t do that. You can also argue that the Kendall Tau correlation has a more intuitive interpretation and easier to calculate, that it gives a better estimate of the corresponding population parameter and that the p values are more accurate in small sample sizes.

Tip add the print() function to see the results of the specific pairwise correlation compututations of columns.

### Plots

#### Summary

| Univariate             | Graphical                               | Non-Graphical                     | 
|-------------|-----------------------------------------|-----------------------------------|
| Categorical | Bar char of frequencies (count / percent) | Frequency/Contingency table (count / percent) |
| Num/Continuous  | Histogram/rugplot/KDE, box/violin/swarm, qqplot, fat tails  | central tendency -mean/median/mode, spread - variance, std, skew, kurt, IQR  |

| Bivariate/multivariate            | Graphical                               | Non-Graphical                     | 
|-------------|-----------------------------------------|-----------------------------------|
| Categorical vs Categorical | heat map, mosaic plot | Two-way Contingency table (count/percent) |
| Continuous vs Continuous  | all pairwise scatterplots, kde, heatmaps |  all pairwise correlation/regression   |
| Categorical vs Continuous  | [bar, violin, swarm, point, strip seaborn plots](http://seaborn.pydata.org/tutorial/categorical.html)  | Summary statistics for each level |

#### Scatter Plots

In [ ]:
plt.scatter(data = df, x = 'num_var1', y = 'num_var2')
# adding a diagonal line through the chart values can help determine trends easily
# You can see if points are above or below the x=y relationship line, therefore if x is higher than y or vice versa
plt.plot([min_value,max_value], [min_value,max_value])

In [ ]:
# regplot combines scatterplot with regression function fitting
sb.regplot(data = df, x = 'num_var1', y = 'num_var2')

In [ ]:
# If we have a very large number of points to plot or our numeric variables are discrete-valued, then it is possible that using a scatterplot straightforwardly will not be informative. The visualization will suffer from overplotting, where the high amount of overlap in points makes it difficult to see the actual relationship between the plotted variables.
# In cases like this, we may want to employ transparency and jitter to make the scatterplot more informative.
# Add Transparency to deal with Overplotting
# Where more points overlap, the darker the image will be.
plt.scatter(data = df, x = 'disc_var1', y = 'disc_var2', alpha = 0.2)

In [ ]:
# Add Jitter to deal with Overplotting
# adding jitter moves the position of each point slightly from its true value
# each point will be plotted in a uniform ±0.2 range of its true values
# jitter won't affect the fit of any regression function, if made
sb.regplot(data = df, x = 'disc_var1', y = 'disc_var2', fit_reg = False,
           x_jitter = 0.2, y_jitter = 0.2, scatter_kws = {'alpha' : 1/3})

#### Heat Maps

In [ ]:
# Visual relationship between the color and density of data
# Add counts cause color perception is imprecise
# Prefer this over a scatterplot if we have 2 discrete variables since the jitter scatterplot can be imprecise. It's also better than adding transparency to a scatterplot in cases where there's a large amount of data.
# However, bin size does affect our interpretation of the data, so make sure to pick the right bin size
# darker values = higher value counts
# specify bin edges for the x and y axis
bins_x = np.arange(0.6, df['disc_var1'].max()+0.4, 0.4)
bins_y = np.arange(0, df['disc_var2'].max()+50, 50)
plt.hist2d(data = df, x = 'disc_var1', y = 'disc_var2',
           bins = [bins_x, bins_y], cmap = 'viridis_r', cmin = 0.5)
plt.colorbar()
plt.xlabel('Displacement (l)')
plt.ylabel('CO2 (g/mi)')

In [ ]:
# Heat Map with counts in each cell
# If you have too many cells in your heat map, then the annotations will end up being too overwhelming,
# too much to attend to. In cases like that, it's best to leave off the annotations and let the data
# and colorbar speak for themselves. You're more likely to see annotations in a categorical heat map,
# where there are going to be fewer cells plotted.
# hist2d returns a number of different variables, including an array of counts
bins_x = np.arange(0.6, df['disc_var1'].max()+0.4, 0.4)
bins_y = np.arange(0, df['disc_var2'].max()+50, 50)
h2d = plt.hist2d(data = df, x = 'disc_var1', y = 'disc_var2',
               bins = [bins_x, bins_y], cmap = 'viridis_r', cmin = 0.5)
counts = h2d[0]
plt.xlabel('Displacement (l)')
plt.ylabel('CO2 (g/mi)')

# loop through the cell counts and add text annotations for each
for i in range(counts.shape[0]):
    for j in range(counts.shape[1]):
        c = counts[i,j]
        if c >= 7: # increase visibility on darkest cells
            plt.text(bins_x[i]+0.5, bins_y[j]+0.5, int(c),
                     ha = 'center', va = 'center', color = 'white')
        elif c > 0:
            plt.text(bins_x[i]+0.5, bins_y[j]+0.5, int(c),
                     ha = 'center', va = 'center', color = 'black')

#### Violin Plot (Categorical vs Numeric)

In [ ]:
# Area depicts the distribution of points. > width = > the number of points
base_color = sns.color_palette()[0]
sns.violinplot(data = df, x = 'cat_var', y = 'num_var', color = base_color)
# switch x and y parameter values to make this a horizontal chart

#### Box Plot (Categorical vs Numeric)

In [ ]:
base_color = sns.color_palette()[0]
sns.boxplot(data = df, x = 'cat_var', y = 'num_var', color = base_color)
# switch x and y parameter values to make this a horizontal chart

#### Clustered Bar Charts (Categorical vs Categorical)

In [ ]:
sb.countplot(data = df, x = 'cat_var1', hue = 'cat_var2')

#### Heat Map (Categorical vs Categorical)

In [ ]:
# Instead of providing the original dataframe, we need to summarize the counts into a matrix that will then be plotted.
ct_counts = df.groupby(['cat_var1', 'cat_var2']).size()
ct_counts = ct_counts.reset_index(name = 'count')
ct_counts = ct_counts.pivot(index = 'cat_var2', columns = 'cat_var1', values = 'count')
sb.heatmap(ct_counts, annot = True, fmt = 'd')
# You can use fmt = '.0f' if you have any cells with no counts, in order to account for NaNs.

#### Faceting (Categorical vs Numeric)

In [ ]:
# Faceting histograms of the quantitative variable, against the qualitative subsets of data
# Facet by col='' parameter
bin_edges = np.arange(-3, df['num_var'].max()+1/3, 1/3)
g = sns.FacetGrid(data = df, col = 'cat_var', size = 2, col_wrap=3)
g.map(plt.hist, "num_var", bins = bin_edges)
g.set_titles('{col_name}')

#### Other Plots

In [ ]:
# compare the means of a feature by different categories of label_colname
df.groupby('label_colname')['afeature'].agg(['count','mean'])

In [ ]:
# plot these means by different categories of label_colname
df['afeature'].groupby(df.label_colname).mean().plot(kind='bar', color=['blue', 'green']) 

In [ ]:
# Scatter plot + print pearson correlation
sns.lmplot('price', 'deal_probability', hue='another_categorical_feature', data=df, fit_reg=True, size=10, aspect=2, scatter_kws={'s': 10, 'alpha':0.3})
plt.xlabel('Price')
plt.ylabel('Deal Probability')
plt.show()
print(f'Pearson correlation : {scipy.stats.pearsonr(df.price.values, df.deal_probability.values)}')

In [ ]:
# Scatter plot with Bokeh (good for large datasets)
from bokeh.charts import Scatter, output_file, show

# Construct the scatter plot
p = Scatter(iris, x='Petal_length', y='Petal_width', color="Class", title="Petal Length vs Petal Width",
            xlabel="Sepal Length", ylabel="Sepal Width")

# Output the file 
output_file('scatter.html')

# Show the scatter plot
show(p)

In [ ]:
# Violin plot -- to know if a categorical_var leans towards to a specific range of numerical_var
sns.violinplot('categorical_var', 'numerical_var', data=df)

# Optional: Trim long-tail/other values and create violin plot again
df_trimmed = df.loc[df.index[df.price < ceiling_value]].sample(n=50000)
sns.violinplot('categorical_var', 'numerical_var', data=df_trimmed)


In [ ]:
# Histograms of different values of categorical vars with numerical var in the same graph
hist_kws={"alpha": 0.2}
sns.distplot(df[df.categorical_var == 'Private']['numerical_var'], label='Private', hist_kws=hist_kws)
sns.distplot(df[df.categorical_var == 'Company']['numerical_var'], label='Company', hist_kws=hist_kws)
sns.distplot(df[df.categorical_var == 'Shop']['numerical_var'], label='Shop', hist_kws=hist_kws)
sns.distplot(df.numerical_var, label='All', hist_kws=hist_kws)
plt.title('numerical_var distribution')
plt.legend()
plt.show()

In [ ]:
# Stacked plot
# libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
 
# Data
r = [0,1,2,3,4]
raw_data = {'greenBars': [20, 1.5, 7, 10, 5], 'orangeBars': [5, 15, 5, 10, 15],'blueBars': [2, 15, 18, 5, 10]}
df = pd.DataFrame(raw_data)
 
# From raw value to percentage
totals = [i+j+k for i,j,k in zip(df['greenBars'], df['orangeBars'], df['blueBars'])]
greenBars = [i / j * 100 for i,j in zip(df['greenBars'], totals)]
orangeBars = [i / j * 100 for i,j in zip(df['orangeBars'], totals)]
blueBars = [i / j * 100 for i,j in zip(df['blueBars'], totals)]
 
# plot
barWidth = 0.85
names = ('A','B','C','D','E')
# Create green Bars
plt.bar(r, greenBars, color='#b5ffb9', edgecolor='white', width=barWidth, label="group A")
# Create orange Bars
plt.bar(r, orangeBars, bottom=greenBars, color='#f9bc86', edgecolor='white', width=barWidth, label="group B")
# Create blue Bars
plt.bar(r, blueBars, bottom=[i+j for i,j in zip(greenBars, orangeBars)], color='#a3acff', edgecolor='white', width=barWidth, label="group C")
 
# Custom x axis
plt.xticks(r, names)
plt.xlabel("group")
 
# Add a legend
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
 
# Show graphic
plt.show()

<img src='stacked_percent_barplot.png'>

# Hypothesis Building

An **independent variable (X)** is the variable that is changed or controlled in a scientific experiment to test the effects on the dependent variable.
A **dependent variable (Y)** is the variable being tested and measured in a scientific experiment.

D is the dependent variable
R is the responding variable
Y is the axis on which the dependent or responding variable is graphed (the vertical axis)

M is the manipulated variable or the one that is changed in an experiment
I is the independent variable
X is the axis on which the independent or manipulated variable is graphed (the horizontal axis)

**STEP 1:** Choose a data set that you would like to work with.

**STEP 2.** Identify a specific topic of interest

**STEP 3.** Prepare a codebook of your own (i.e., print individual pages or copy screen and paste into a new document) from the larger codebook that includes the questions/items/variables that measure your selected topics.)

**STEP 4.** Identify a second topic that you would like to explore in terms of its association with your original topic.

**STEP 5.** Add questions/items/variables documenting this second topic to your personal codebook.

**STEP 6.** Identify a basic research question that interests you. - "Is there an association between A and B
in subset of the population C"?

**STEP 7.** Once you have a basic research question, you can think about other variables that might explain your association.

**STEP 8.** Perform a literature review to see what research has been previously done on this topic. Use sites such as Google Scholar ([http://scholar.google.com](http://scholar.google.com/)) to search for published academic work in the area(s) of interest. Try to find multiple sources, and take note of basic bibliographic information.

**STEP 9.** Based on your literature review, develop a hypothesis about what you believe the association might be between these topics. Be sure to integrate the specific variables you selected into the hypothesis.

**STEP 10.** Ddscribe what you intend to do with your variables (data management/feature creation). For example, do I want to
look at how often adolescents binge drink, or do I want to create a binary variables (yes/no) for whether adolescents binge drank in the past month? Do I want depression to be a quantitative variable that aggregates multiple questions that ask about depression symptoms, or do I want to use a cutoff to create a categorical variable that groups adolescents into low, medium, or high levels of depression? These decisions should be described when you describe the variables you will be using.

All of this is a starting point for your analysis and will change as you perform the analysis.

# Writing About Data

<img src="How-to-Write-About-Your-Data1.png">

<img src="How-to-Write-About-Your-Data2.png">

Example:

Sample

The sample is from the first wave of the National Epidemiologic Survey on Alcohol and
Related Conditions (NESARC), the largest nationwide longitudinal survey of alcohol and
drug use and associated psychiatric and medical comorbidities. Participants (N=43,093)
represented the civilian, non-institutionalized adult population of the United States, and
included persons living in households, military personnel living off base, and persons
residing in the following group quarters: boarding or rooming houses, non-transient hotels
and motels, shelters, facilities for housing workers, college quarters, and group homes. The
NESARC included over sampling of Blacks, Hispanics and young adults aged 18 to 24 years.
The data analytic sample for this study included participants 18-25 years old who reported
smoking at least 1 cigarette per day in the past 30 days (N=1,320).

Procedure

Data were collected by trained U.S. Census Bureau Field Representatives during 2001–
2002 through computer-assisted personal interviews (CAPI). One adult was selected for
interview in each household, and interviews were conducted in respondents’ homes
following informed consent procedures.

Measures

Lifetime major depression (i.e. those experienced in the past 12 months and prior to the
past 12 months) was assessed using the NIAAA, Alcohol Use Disorder and Associated
Disabilities Interview Schedule – DSM-IV (AUDADIS-IV) (Grant et al., 2003; Grant, Harford,
Dawson, & Chou, 1995). The tobacco module of the AUDADIS-IV contains detailed
questions on the frequency, quantity, and patterning of tobacco use as well as symptom
criteria for DSM-IV nicotine dependence. Current smoking was evaluated through both
smoking frequency (“About how often did you usually smoke in the past year?”) coded
dichotomously to represent presence or absence of daily smoking, and quantity (“On the
days that you smoked in the last year, about how many cigarettes did you usually smoke?”),
a quantitative variable that ranged from 1 cigarette per day to 98 cigarettes per day.

# Statistical Analysis

<img src="wrangling-notebook-15.png">

### P values

<img src="Mind-Map-p-values.png">

<img src="Mind-Map-p-values-scale.png">

### ANOVA Test
Categorical → Quantitative

- Goal: A (C) is correlated to the level of B (Q) i.e. Examine the differences in the means of B across the categories of A.
- Null Hypothesis: The means across the categories of A are equal.
- Alternate Hypothesis: The means across categories of A are ≠.
- There are many ways for the population means to not be equal.

<img src="wrangling-notebook-16.png">

<img src="Mind-Map-ANOVA-scale.png">

In [ ]:
import numpy
import pandas
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

data = pandas.read_csv('nesarc.csv', low_memory=False)

#setting variables you will be working with to numeric
data['S3AQ3B1'] = pandas.to_numeric(data['S3AQ3B1'], errors='coerce')
data['S3AQ3C1'] = pandas.to_numeric(data['S3AQ3C1'], errors='coerce')
data['CHECK321'] = pandas.to_numeric(data['CHECK321'], errors='coerce')

#subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1=data[(data['AGE']>=18) & (data['AGE']<=25) & (data['CHECK321']==1)]

#SETTING MISSING DATA
sub1['S3AQ3B1']=sub1['S3AQ3B1'].replace(9, numpy.nan)
sub1['S3AQ3C1']=sub1['S3AQ3C1'].replace(99, numpy.nan)

# ADDITIONAL DATA WRANGLING
#recoding number of days smoked in the past month
recode1 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub1['USFREQMO']= sub1['S3AQ3B1'].map(recode1)

#converting new variable USFREQMMO to numeric
data['USFREQMO'] = pandas.to_numeric(data['USFREQMO'], errors='coerce')

# Creating a secondary variable multiplying the days smoked/month and the number of cig/per day
sub1['NUMCIGMO_EST']=sub1['USFREQMO'] * sub1['S3AQ3C1']

data['NUMCIGMO_EST'] = pandas.to_numeric(data['NUMCIGMO_EST'], errors='coerce')

ct1 = sub1.groupby('NUMCIGMO_EST').size()
print (ct1)


# ANOVA Testing
# ols - ordinary least squares
# using ols function for calculating the F-statistic and associated p value
# Calculate differences in the mean num of response var B (NUMCIGMO_EST), for cases with and without explanatory var A (MAJORDEPLIFE)
# C() indicates categorical variable

# ---- For categorical variables with 2 Groups
sub1 = df
model1 = smf.ols(formula='NUMCIGMO_EST ~ C(MAJORDEPLIFE)', data=sub1)
results1 = model1.fit()
# Generates F-statistic and p-value which is called Prob (F-statistic) in the output
# We look at the P-value here to determine if correlation exists or not (< 0.5 correlation exists, otherwise no)
print (results1.summary())

# Create a subset of the dataset and calculate the means of B, dropping na to get only non-null values, because the OLS analysis also includes only non-null values
# We get the means and std to say that these means are statistically equal or not (depending on the p-value)
# If p-value < 0.5, by looking at the means tables, we can see which category (presence of A or its absence) resulted in more B.

sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()

print ('means for numcigmo_est by major depression status')
m1= sub2.groupby('MAJORDEPLIFE').mean()
print (m1)

print ('standard deviations for numcigmo_est by major depression status')
sd1 = sub2.groupby('MAJORDEPLIFE').std()
print (sd1)

# ---- For categorical variables with more than 2 Groups
sub3 = sub1[['NUMCIGMO_EST', 'ETHRACE2A']].dropna()

model2 = smf.ols(formula='NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub3).fit()
print (model2.summary())

print ('means for numcigmo_est by major depression status')
m2= sub3.groupby('ETHRACE2A').mean()
print (m2)

print ('standard deviations for numcigmo_est by major depression status')
sd2 = sub3.groupby('ETHRACE2A').std()
print (sd2)

# Post-Hoc Tests
# Help you figure out which categories are similar in dependence
# F-test and p-value don't provide insight into why the null hypothesis can be rejected because there are multiple levels to the categorical explanatory variable
# They do not tell us in what way the population means are not statistically equal.
# There are many ways for population means not to be all equal.
# Maybe they are not all equal to each other. Maybe only 2 of the population means are not equal to each other, but the rest are. 

# ANOVA doesn't tell us which groups are different. To determine that, we need or perform a post-hoc test (conducts post-hoc paired comparisons)
# Performs paired post-hoc ANOVA testing in a way to prevent Type One Error (rejecting the null hypothesis when the null hypothesis is true)
# We can't manually pair different categories and do ANOVA testing on them because there's a ahcnace of making a 5% error on each analysis that we conduct (cause our p value ceiling is 0.05). Our overall chance of committing type 1 error can be far > 5%. For e,g, after 15 tests, it is upto 54%. This error rate for group pair comparison is called the family wise error rate.
# Post-hoc tests are designed to evaluate the difference between pairs of means while protecting against inflation of Type 1 errors.
# There's a lot of post-hoc tests for ANOVA to choose from, they differ in their conservativeness of preventing type 1 error inflation.
# We use Tukey's Honestly Significant Difference Test
# multi.MultiComparison(quantitative response var, categorical explanatory var)
# the last column (reject) shows pairs in which we can reject the null hypothesis. True = there is difference in B for these different categories of A.
# Type 1 Error: When we think there is correlation, when there's actually no correlation.
mc1 = multi.MultiComparison(sub3['NUMCIGMO_EST'], sub3['ETHRACE2A'])
res1 = mc1.tukeyhsd()
print(res1.summary())

Example Explanation of ANOVA results from code above:

**Model Interpretation for ANOVA:**

When examining the association between current number of B (quantitative response) and A (categorical explanatory), an Analysis of Variance (ANOVA) revealed that among daily, B of type T (my sample), those with A reported significantly more B (Mean=14.6, s.d. ±9.15) compared to those without A (Mean=11.4, s.d. ±7.43), F(1, 1313)=44.68, p<0001.

Note that the degrees of freedom that I report in parentheses) following ‘F’ can be found in the OLS table as the DF model and DF residuals. In this example 44.68 is the actual F value from the OLS table and we commonly report a very small p value as simply <.0001.

**Model Interpretation for post hoc ANOVA results:**

ANOVA revealed that among daily, B of type T (my sample), A (collapsed into 5 ordered categories, which is the categorical explanatory variable) and B (quantitative response variable) were significantly associated, F (4, 1308)=11.79, p=0001. Post hoc comparisons of mean number of A by pairs of B per day categories revealed that those individuals with B more than 10 per day (i.e. 11 to 15, 16 to 20 and >20) reported significantly more A compared to those with B of 10 or fewer per day (i.e. 1 to 5 and 6 to 10). All other comparisons were statistically similar.

### Chi-Squared Test

- Categorical explanatory → Categorical response vars
- Can be used for 2 or more categories.
- Step 1: Which variable plays the role of the explanatory variable
- Step 2: Calculate conditional percentages separately (% of the response variable for each value of the explanatory variable).
- Step 3: Is it statistically significant?
  - Null Hypothesis: No relationship between the 2 categorical variables, independent e.g. genders and drinking (difference is due to sample variability).
  - Alternate Hypothesis: Correlation between the 2 categorical variables, not indepedent e.g. genders and drinking (difference is due to gender).
- The idea behind the Chi-squared test of independence is to measure how far the data is from what is claimed in the null hypothesis.
- We'll have 2 sets of counts:
  - **The Observed Counts**: The data as is
  - **The Expected Counts**: To represent data if there was no correlation between A and B, and the null hypothesis were true.
    - Calculate the counts we'd expect to see if both the explanatory variable and response variable were independent, and the null hypothesis is true.
    - P(A and B) = P(A) * P(B) (given A & B are independent - which we are assuming here)
    - This is the probability of one instance of A and B.
    - The probability of A and B happening N times is = N*P(A and B)
    - Measure how far the observed counts are from the expected counts.
  - We'll base our decision on the size of the discrepancy between what we observe and what we would expect to observe if the null hypothesis was true.
- The Chi-Squared test tells us how far the observed data is from the expected data, if the null hypothesis was true.
  - For a 2 by 2, the cutoff limit is 3.84. Chi-square score > 3.84 is considered large.
  - For cases other than 2 by 2s, the cutoff limit is determined by the null distribution in that case.
  - Therefore, we don't use the chi-square statistic, and only rely on the p-value.
  - The p-value of a chi-squared test is the probability of observing a chi-square score at least as large as the one observed.

In [ ]:
import pandas
import numpy
import scipy.stats
import seaborn
import matplotlib.pyplot as plt

data = pandas.read_csv('nesarc.csv', low_memory=False)

# new code setting variables you will be working with to numeric
data['TAB12MDX'] = pandas.to_numeric(data['TAB12MDX'], errors='coerce')
data['CHECK321'] = pandas.to_numeric(data['CHECK321'], errors='coerce')
data['S3AQ3B1'] = pandas.to_numeric(data['S3AQ3B1'], errors='coerce')
data['S3AQ3C1'] = pandas.to_numeric(data['S3AQ3C1'], errors='coerce')
data['AGE'] = pandas.to_numeric(data['AGE'], errors='coerce')

#subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1=data[(data['AGE']>=18) & (data['AGE']<=25) & (data['CHECK321']==1)]

#make a copy of my new subsetted data
sub2 = sub1.copy()

# recode missing values to python missing (NaN)
sub2['S3AQ3B1']=sub2['S3AQ3B1'].replace(9, numpy.nan)
sub2['S3AQ3C1']=sub2['S3AQ3C1'].replace(99, numpy.nan)

#recoding values for S3AQ3B1 into a new variable, USFREQMO
recode1 = {1: 30, 2: 22, 3: 14, 4: 6, 5: 2.5, 6: 1}
sub2['USFREQMO']= sub2['S3AQ3B1'].map(recode1)

# contingency table of observed counts
ct1=pandas.crosstab(sub2['TAB12MDX'], sub2['USFREQMO'])
print (ct1)

# column percentages
colsum=ct1.sum(axis=0)
colpct=ct1/colsum
print(colpct)

# chi-square
print ('chi-square value, p value, expected counts')
cs1= scipy.stats.chi2_contingency(ct1)
print (cs1)


# We calculate column percentages because:
# + Row %: all rows add up to 100%
# + Cell %: all cells in the table together add up to 100%
# + Column %: all column add up to 100%
'''
We calculate column percentages to determine the Chi-Squared test results because
the explanatory variable categories are represented as different columns,
and response variable values are represented as rows,
it'll be the explanatory variable categories(i.e. columns) that we want to interpret.
i.e. we are interested in how the rate of response variable differs according to which
explanatory group the observations belong to.
We're not interested in the column percentages in observations without dependence.

'''
# If our explanatory variable had only 2 categories, we could interpret the 2 corresponding column percentages
# and be able to say which group had a significantly higher rate of dependence.
# In case of more than one category, we know that not all are equal,
# but don't know which are different and which are not.

# Plot the frequency of different categories of explanatory variable vs % of response variable as bar charts.
# See next cell for plot
# set variable types 
sub2["USFREQMO"] = sub2["USFREQMO"].astype('category')
# new code for setting variables to numeric:
sub2['TAB12MDX'] = pandas.to_numeric(sub2['TAB12MDX'], errors='coerce')

# graph percent with nicotine dependence within each smoking frequency group 
seaborn.factorplot(x="USFREQMO", y="TAB12MDX", data=sub2, kind="bar", ci=None)
plt.xlabel('Days smoked per month')
plt.ylabel('Proportion Nicotine Dependent')

# Same post-hoc principles as in ANOVA test. Same principles of avoiding Type 1 errors.
# Using Bonferroni Adjustment: p-value/c (number of comparisons we plan to make)
# We need to then run a Chi-Square test for each of the paired comparisons.
# If your explanatory variable has more than two levels or groups, you'll also need to conduct
# a post hoc test. We use the Bonferroni Adjustment to protect against Type I error,
# and then run the Chi Square Test of Independence for each paired comparison.

# Create new variables that allow you to choose only 2 categories at a time. We use recodes with the map func to do this
# We keep only 2 values (1 and 2.5) and remove the rest.
# We create a contingency table, get col %s and a new chi-square table with only these 2 variables
# If the p-value is greater than the Bonferroni Adjusted p-value, we know there is no correlation.
recode2 = {1: 1, 2.5: 2.5}
sub2['COMP1v2']= sub2['USFREQMO'].map(recode2)

# contingency table of observed counts
ct2=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP1v2'])
print (ct2)

# column percentages
colsum=ct2.sum(axis=0)
colpct=ct2/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs2= scipy.stats.chi2_contingency(ct2)
print (cs2)

# ---- We finish by doing the same thing above for each of the other pairs
recode3 = {1: 1, 6: 6}
sub2['COMP1v6']= sub2['USFREQMO'].map(recode3)

# contingency table of observed counts
ct3=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP1v6'])
print (ct3)

# column percentages
colsum=ct3.sum(axis=0)
colpct=ct3/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs3= scipy.stats.chi2_contingency(ct3)
print (cs3)

recode4 = {1: 1, 14: 14}
sub2['COMP1v14']= sub2['USFREQMO'].map(recode4)

# contingency table of observed counts
ct4=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP1v14'])
print (ct4)

# column percentages
colsum=ct4.sum(axis=0)
colpct=ct4/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs4= scipy.stats.chi2_contingency(ct4)
print (cs4)

recode5 = {1: 1, 22: 22}
sub2['COMP1v22']= sub2['USFREQMO'].map(recode5)

# contingency table of observed counts
ct5=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP1v22'])
print (ct5)

# column percentages
colsum=ct5.sum(axis=0)
colpct=ct5/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs5= scipy.stats.chi2_contingency(ct5)
print (cs5)

recode6 = {1: 1, 30: 30}
sub2['COMP1v30']= sub2['USFREQMO'].map(recode6)

# contingency table of observed counts
ct6=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP1v30'])
print (ct6)

# column percentages
colsum=ct6.sum(axis=0)
colpct=ct6/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs6= scipy.stats.chi2_contingency(ct6)
print (cs6)

recode7 = {2.5: 2.5, 6: 6}
sub2['COMP2v6']= sub2['USFREQMO'].map(recode7)

# contingency table of observed counts
ct7=pandas.crosstab(sub2['TAB12MDX'], sub2['COMP2v6'])
print (ct7)

# column percentages
colsum=ct7.sum(axis=0)
colpct=ct7/colsum
print(colpct)

print ('chi-square value, p value, expected counts')
cs7=scipy.stats.chi2_contingency(ct7)
print (cs7)

Example Interpretation of Chi-Square Tests results from code above:

When examining the association between lifetime major depression (categorical response) and past year nicotine dependence (categorical explanatory), a chi-square test of independence revealed that among daily, young adults smokers (my sample), those with past year nicotine dependence were more likely to have experienced major depression in their lifetime(36.2%) compared to those without past year nicotine dependence (12.7%), X2 =88.60, 1 df, p=0001.

The df or degree of freedom we record is the number of levels of the explanatory variable -1. Here the df is 1 nicotine dependence which has 2 levels (df 2-1=1).

Model Interpretation for post hoc Chi-Square Test results:

A Chi Square test of independence revealed that among daily, young adult smokers (my sample), number of cigarettes smoked per day (collapsed into 5 ordered categories) and past year

nicotine dependence (binary categorical variable) were significantly associated, X2 =45.16, 4 df, p=.0001.

Post hoc comparisons of rates of nicotine dependence by pairs of cigarettes per day categories revealed that higher rates of nicotine dependence were seen among those smoking more cigarettes, up to 11 to 15 cigarettes per day. In comparison, prevalence of nicotine dependence was statistically similar among those groups smoking 10 to 15, 16 to 20, and > 20 cigarettes per day.

Chart generated from above code.
<img src="wrangling-notebook-18.png">

**Explanation of Chi-Square Test**
One way to plot the similarities: Use letters to denote categories where depdendent variable rates are similar.
<img src="wrangling-notebook-19.png">



### Pearson Correlation

### Power Analysis